In [32]:
import numpy as np

In [33]:
!python --version

Python 3.11.5


In [34]:
#demo for evaluation
#https://gtuckerkellogg.github.io/pairwise/demo/

In [35]:
""" 
F(i, j) = max(
            F(i - 1, j - 1) + s(xi, yj),
            maxk=0…i-1F(k, j) - γ(i - k),
            maxk=0…j-1F(i, k) - γ(j - k),
            0)   

"""

' \nF(i, j) = max(\n            F(i - 1, j - 1) + s(xi, yj),\n            maxk=0…i-1F(k, j) - γ(i - k),\n            maxk=0…j-1F(i, k) - γ(j - k),\n            0)   \n\n'

In [36]:
scores = {
    "match" : 1,
    "mismatch" : -1,
    "gap": 2
}

def s(xi,xj):
    return scores["match"] if xi == xj else scores["mismatch"]

def match_mismatch(i,j, xi, xj, matrix):
    return matrix[i-1,j-1] + s(xi, xj)

#maxk=0…i-1F(k, j) - γ(i - k)
def γ(n):
    return n * scores["gap"]

def row_gaps_score(i,j):
    gap_scores = []
    for k in range((i)):
        gap_score = matrix[k,j] - γ(i-k)
        gap_scores.append(gap_score)
    
    max_score = np.max(gap_scores)
    kth_rows = tuple([k for k,value in enumerate(gap_scores) if value==max_score])
    #print(kth_rows)
    return max_score, kth_rows

#maxk=0…j-1F(i, k) - γ(j - k),
def col_gaps_score(i,j):
    gap_scores = []
    for k in range((j)):
        gap_score = matrix[i,k] - γ(j-k)
        gap_scores.append(gap_score)

    max_score = np.max(gap_scores)
    kth_cols = tuple([k for k,value in enumerate(gap_scores) if value==max_score])
    #print(kth_cols)
    return max_score, kth_cols

bottom_sequence = "PAWHEAE"
top_sequence = "HEAGAWGHEE"
n_rows = len(bottom_sequence)+1
n_cols = len(top_sequence)+1

matrix = np.zeros((n_rows, n_cols))
traceback_dic = {}

for i in range(1,n_rows):
    for j in range(1,n_cols):
        #print(f"i: {i}, j: {j}")
        xi = bottom_sequence[i-1]
        xj = top_sequence[j-1]
        score1 = match_mismatch(i,j, xi, xj, matrix)
        score2, kth_rows = row_gaps_score(i,j)
        score3, kth_cols = col_gaps_score(i,j)
        top_score = max(score1, 
                        score2, 
                        score3, 
                        0)
        
        matrix[i,j] = top_score

        top_idx = [t for t, value in enumerate([score1, score2, score3]) if value == top_score]
        traceback_dic[f"{i},{j}"] = []
        for idx in top_idx:
            #print(f"i,j: {i},{j}: ",end=" ")
            #print(traceback_dic[f"{i},{j}"])
            if top_score > 0:
                if idx == 0:
                    traceback_dic[f"{i},{j}"].append((i-1,j-1))
                elif idx == 1:
                    for k in kth_rows:
                        traceback_dic[f"{i},{j}"].append((k,j))
                elif idx == 2:
                    for k in kth_cols:
                        traceback_dic[f"{i},{j}"].append((i,k))                

traceback_dic

{'1,1': [],
 '1,2': [],
 '1,3': [],
 '1,4': [],
 '1,5': [],
 '1,6': [],
 '1,7': [],
 '1,8': [],
 '1,9': [],
 '1,10': [],
 '2,1': [],
 '2,2': [],
 '2,3': [(1, 2)],
 '2,4': [],
 '2,5': [(1, 4)],
 '2,6': [],
 '2,7': [],
 '2,8': [],
 '2,9': [],
 '2,10': [],
 '3,1': [],
 '3,2': [],
 '3,3': [],
 '3,4': [],
 '3,5': [],
 '3,6': [(2, 5)],
 '3,7': [],
 '3,8': [],
 '3,9': [],
 '3,10': [],
 '4,1': [(3, 0)],
 '4,2': [],
 '4,3': [],
 '4,4': [],
 '4,5': [],
 '4,6': [],
 '4,7': [(3, 6)],
 '4,8': [(3, 7)],
 '4,9': [],
 '4,10': [],
 '5,1': [],
 '5,2': [(4, 1)],
 '5,3': [],
 '5,4': [],
 '5,5': [],
 '5,6': [],
 '5,7': [],
 '5,8': [],
 '5,9': [(4, 8)],
 '5,10': [(4, 9)],
 '6,1': [],
 '6,2': [],
 '6,3': [(5, 2)],
 '6,4': [(6, 3)],
 '6,5': [(5, 4)],
 '6,6': [],
 '6,7': [],
 '6,8': [],
 '6,9': [],
 '6,10': [(5, 9)],
 '7,1': [],
 '7,2': [(6, 1)],
 '7,3': [(6, 3)],
 '7,4': [(6, 3)],
 '7,5': [],
 '7,6': [],
 '7,7': [],
 '7,8': [],
 '7,9': [(6, 8)],
 '7,10': [(6, 9)]}

In [37]:
import pandas as pd

# Define the shape of the matrix
rows, cols = 7, 10

# Create the matrix with the specified entries
visual_matrix = [[f"({row},{col})" for col in range(cols)] for row in range(rows)]

# Convert to a pandas DataFrame
matrix_df = pd.DataFrame(visual_matrix)

# Display the DataFrame
print(matrix_df)


       0      1      2      3      4      5      6      7      8      9
0  (0,0)  (0,1)  (0,2)  (0,3)  (0,4)  (0,5)  (0,6)  (0,7)  (0,8)  (0,9)
1  (1,0)  (1,1)  (1,2)  (1,3)  (1,4)  (1,5)  (1,6)  (1,7)  (1,8)  (1,9)
2  (2,0)  (2,1)  (2,2)  (2,3)  (2,4)  (2,5)  (2,6)  (2,7)  (2,8)  (2,9)
3  (3,0)  (3,1)  (3,2)  (3,3)  (3,4)  (3,5)  (3,6)  (3,7)  (3,8)  (3,9)
4  (4,0)  (4,1)  (4,2)  (4,3)  (4,4)  (4,5)  (4,6)  (4,7)  (4,8)  (4,9)
5  (5,0)  (5,1)  (5,2)  (5,3)  (5,4)  (5,5)  (5,6)  (5,7)  (5,8)  (5,9)
6  (6,0)  (6,1)  (6,2)  (6,3)  (6,4)  (6,5)  (6,6)  (6,7)  (6,8)  (6,9)


In [38]:
matrix

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0.],
       [0., 0., 2., 0., 0., 0., 0., 0., 0., 2., 1.],
       [0., 0., 0., 3., 1., 1., 0., 0., 0., 0., 1.],
       [0., 0., 1., 1., 2., 0., 0., 0., 0., 1., 1.]])